### Project Solution - Goal 1

In [2]:
file_name = 'nyc_parking_tickets_extract.csv'

In [3]:
with open(file_name) as f:
    for _ in range(5):
        print(next(f))

Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Violation Description

4006478550,VAD7274,VA,PAS,10/5/2016,5,4D,BMW,BUS LANE VIOLATION

4006462396,22834JK,NY,COM,9/30/2016,5,VAN,CHEVR,BUS LANE VIOLATION

4007117810,21791MG,NY,COM,4/10/2017,5,VAN,DODGE,BUS LANE VIOLATION

4006265037,FZX9232,NY,PAS,8/23/2016,5,SUBN,FORD,BUS LANE VIOLATION



In [9]:
with open(file_name) as f:
    column_headers = next(f).strip('\n').split(',')
    sample_data = next(f).strip('\n').split(',')

In [10]:
column_headers

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Description']

In [11]:
sample_data

['4006478550',
 'VAD7274',
 'VA',
 'PAS',
 '10/5/2016',
 '5',
 '4D',
 'BMW',
 'BUS LANE VIOLATION']

In [14]:
column_names = [header.replace(' ', '_').lower() for header in column_headers]

In [15]:
column_names

['summons_number',
 'plate_id',
 'registration_state',
 'plate_type',
 'issue_date',
 'violation_code',
 'vehicle_body_type',
 'vehicle_make',
 'violation_description']

In [16]:
list(zip(column_names, sample_data))

[('summons_number', '4006478550'),
 ('plate_id', 'VAD7274'),
 ('registration_state', 'VA'),
 ('plate_type', 'PAS'),
 ('issue_date', '10/5/2016'),
 ('violation_code', '5'),
 ('vehicle_body_type', '4D'),
 ('vehicle_make', 'BMW'),
 ('violation_description', 'BUS LANE VIOLATION')]

- 'summons_number' - int  <--
- 'plate_id' - str
- 'registration_state' - str
- 'plate_type' - str
- 'issue_date' - date  <--
- 'violation_code' - int  <--
- 'vehicle_body_type' - str
- 'vehicle_make' - str
- 'violation_description' - str

In [17]:
from collections import namedtuple
Ticket = namedtuple('Ticket', column_names)

In [18]:
with open(file_name) as f:
    next(f)
    raw_data_row = next(f)

In [19]:
raw_data_row

'4006478550,VAD7274,VA,PAS,10/5/2016,5,4D,BMW,BUS LANE VIOLATION\n'

In [20]:
def read_data():
    with open(file_name) as f:
        next(f)
        yield from f

In [21]:
raw_data = read_data()

In [22]:
for _ in range(5):
    print(next(raw_data))

4006478550,VAD7274,VA,PAS,10/5/2016,5,4D,BMW,BUS LANE VIOLATION

4006462396,22834JK,NY,COM,9/30/2016,5,VAN,CHEVR,BUS LANE VIOLATION

4007117810,21791MG,NY,COM,4/10/2017,5,VAN,DODGE,BUS LANE VIOLATION

4006265037,FZX9232,NY,PAS,8/23/2016,5,SUBN,FORD,BUS LANE VIOLATION

4006535600,N203399C,NY,OMT,10/19/2016,5,SUBN,FORD,BUS LANE VIOLATION



In [23]:
def parse_int(value, *, default=None):
    try:
        return int(value)
    except ValueError:
        return default

In [24]:
from datetime import datetime

In [25]:
def parse_date(value, *, default=None):
    date_format = '%m/%d/%Y'
    try:
        return datetime.strptime(value, date_format).date()
    except ValueError:
        return default

In [26]:
parse_int('hello', default='N/A')

'N/A'

In [27]:
parse_date('3/28/2018')

datetime.date(2018, 3, 28)

In [28]:
parse_date('234556', default='N/A')

'N/A'

In [29]:
def parse_string(value, *, default=None):
    try:
        cleaned = value.strip()
        if not cleaned:
            return default
        else:
            return cleaned
    except ValueError:
        return default

In [30]:
parse_string('   hello  ')

'hello'

In [31]:
parse_string('  ', default='N/A')

'N/A'

In [ ]:
column_names

['summons_number',
 'plate_id',
 'registration_state',
 'plate_type',
 'issue_date',
 'violation_code',
 'vehicle_body_type',
 'vehicle_make',
 'violation_description']

In [34]:
from functools import partial

column_parsers = (parse_int,
                  parse_string,
                  lambda x: parse_string(x, default=''),  # both work the same way, passing a function with an argument
                  partial(parse_string, default=''),  # both work the same way, passing a function with an argument
                  parse_date,
                  parse_int,
                  partial(parse_string, default=''),
                  parse_string,
                  lambda x: parse_string(x, default=''))


In [35]:
def parse_row(row):
    fields = row.strip('\n').split(',')
    parsed_data = (func(field)
                   for func, field in zip(column_parsers, fields))
    return parsed_data

In [37]:
rows = read_data()
for _ in range(5):
    row = next(rows)
    parsed_data = parse_row(row)
    print(list(parsed_data))

[4006478550, 'VAD7274', 'VA', 'PAS', datetime.date(2016, 10, 5), 5, '4D', 'BMW', 'BUS LANE VIOLATION']
[4006462396, '22834JK', 'NY', 'COM', datetime.date(2016, 9, 30), 5, 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
[4007117810, '21791MG', 'NY', 'COM', datetime.date(2017, 4, 10), 5, 'VAN', 'DODGE', 'BUS LANE VIOLATION']
[4006265037, 'FZX9232', 'NY', 'PAS', datetime.date(2016, 8, 23), 5, 'SUBN', 'FORD', 'BUS LANE VIOLATION']
[4006535600, 'N203399C', 'NY', 'OMT', datetime.date(2016, 10, 19), 5, 'SUBN', 'FORD', 'BUS LANE VIOLATION']
